# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

### Get data from `.csv` files and then preprocess data

In [2]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    df_eng['Original Language'] = 'english'
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_fr['Original Language'] = 'french'
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    df_sp['Original Language'] = 'spanish'
    
    df_fr.columns =  df_eng.columns
    df_sp.columns = df_eng.columns
    
    
    df = pd.concat([df_eng, df_fr, df_sp], axis=0, ignore_index=True).reset_index()
    df['Review'] = df.Review.str.replace('"', '')

    return df

df = preprocess_data()

In [3]:
df.sample(10)

,index,Movie / TV Series,Year,Synopsis,Review,Original Language
24,24,Amores perros,2000,Tres historias se entrelazan en esta película ...,Amores perros es una película intensa y conmov...,spanish
19,19,Babylon A.D.,2008,"Dans un futur lointain, un mercenaire doit esc...",Ce film est un gâchis complet. Les personnages...,french
3,3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,The Godfather is a classic movie that stands t...,english
9,9,The Island,2005,In a future where people are cloned for organ ...,The Island is a bland and forgettable sci-fi t...,english
25,25,Águila Roja,(2009-2016),Esta serie de televisión española sigue las av...,Águila Roja es una serie aburrida y poco inter...,spanish
1,1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,The Dark Knight is a thrilling and intense sup...,english
20,20,Roma,2018,Cleo (Yalitza Aparicio) es una joven empleada ...,Roma es una película hermosa y conmovedora que...,spanish
18,18,Les Visiteurs en Amérique,2000,Dans cette suite de la comédie française Les V...,Le film est une perte de temps totale. Les bla...,french
10,10,La La Land,2016,Cette comédie musicale raconte l'histoire d'un...,La La Land est un film absolument magnifique a...,french
16,16,La Tour Montparnasse Infernale,2001,Deux employés de bureau incompétents se retrou...,Je ne peux pas croire que j'ai perdu du temps ...,french


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [4]:
from transformers import MarianMTModel, MarianTokenizer


fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

/opt/venv/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [5]:
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """    
    # encode the text using the tokenizer
    inputs = tokenizer.batch_encode_plus(text, return_tensors="pt", padding=True)
    
    # generate the translation using the model
    outputs = model.generate(inputs['input_ids'], max_new_tokens=512)

    # decode the generated output and return the translated text
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return decoded

In [6]:
# filter reviews in French and translate to English
fr_reviews = df[df["Original Language"] == 'french'].Review.to_list()
fr_reviews_en = translate(fr_reviews, fr_en_model, fr_en_tokenizer)

# filter synopsis in French and translate to English
fr_synopsis = df[df["Original Language"] == 'french'].Synopsis.to_list()
fr_synopsis_en = translate(fr_synopsis, fr_en_model, fr_en_tokenizer)

# filter reviews in Spanish and translate to English
es_reviews = df[df["Original Language"] == 'spanish'].Review.to_list()
es_reviews_en = translate(es_reviews, es_en_model, es_en_tokenizer)

# filter synopsis in Spanish and translate to English
es_synopsis = df[df["Original Language"] == 'spanish'].Synopsis.to_list()
es_synopsis_en = translate(es_synopsis, es_en_model, es_en_tokenizer)


In [7]:
# update dataframe with translated text
# add the translated reviews and synopsis

fr_reviews = fr_reviews_en
df.loc[df["Original Language"] == 'french', 'Review'] = fr_reviews

fr_synopsis = fr_synopsis_en
df.loc[df["Original Language"] == 'french', 'Synopsis'] = fr_synopsis

es_reviews = es_reviews_en
df.loc[df["Original Language"] == 'spanish', 'Review'] = es_reviews

es_synopsis = es_synopsis_en
df.loc[df["Original Language"] == 'spanish', 'Synopsis'] = es_synopsis

In [8]:
df.drop(columns='index', inplace=True)

In [9]:
df.sample(10)

,Movie / TV Series,Year,Synopsis,Review,Original Language
27,El Bar,2017,A group of people are trapped in a bar after M...,The Bar is a ridiculous and meaningless film t...,spanish
1,The Dark Knight,2008,Batman (Christian Bale) teams up with District...,The Dark Knight is a thrilling and intense sup...,english
16,La Tour Montparnasse Infernale,2001,Two incompetent office workers find themselves...,I can't believe I've wasted time watching this...,french
18,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,The film is a total waste of time. The jokes a...,french
9,The Island,2005,In a future where people are cloned for organ ...,The Island is a bland and forgettable sci-fi t...,english
21,La Casa de Papel,(2017-2021),This Spanish television series follows a group...,The Paper House is an exciting and addictive s...,spanish
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,It was difficult to sit through the whole thin...,english
11,Intouchables,2011,This film tells the story of the unlikely frie...,Untouchables is an incredibly touching film wi...,french
20,Roma,2018,Cleo (Yalitza Aparicio) is a young domestic wo...,Rome is a beautiful and moving film that pays ...,spanish
12,Amélie,2001,This romantic comedy tells the story of Amélie...,Amélie is an absolutely charming film that wil...,french


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
model_name = "distilbert-base-uncased"
sentiment_classifier = pipeline('text-classification', model=model_name) #"automatic-speech-recognition", model=model_name)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

In [14]:
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    return classifier(text)

In [15]:
# perform sentiment analysis on reviews and store results in new column
sentiment = pd.DataFrame(analyze_sentiment(df.Review.tolist(), sentiment_classifier))

In [16]:
df = pd.concat([df, sentiment], axis=1)
df.sample(10)

,Movie / TV Series,Year,Synopsis,Review,Original Language,label,score
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...",Torrente is a vulgar and offensive film that a...,spanish,NEGATIVE,0.999751
20,Roma,2018,Cleo (Yalitza Aparicio) is a young domestic wo...,Rome is a beautiful and moving film that pays ...,spanish,POSITIVE,0.999882
12,Amélie,2001,This romantic comedy tells the story of Amélie...,Amélie is an absolutely charming film that wil...,french,POSITIVE,0.999892
17,Astérix aux Jeux Olympiques,2008,In this film adaptation of the popular comic s...,This film is a complete disappointment. The jo...,french,NEGATIVE,0.999774
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"Dull and pointless, with none of the magic of ...",english,NEGATIVE,0.999795
23,El Laberinto del Fauno,2006,"During the Spanish postwar period, Ofelia (Iva...",The Labyrinth of Fauno is a fascinating and em...,spanish,POSITIVE,0.999873
10,La La Land,2016,This musical tells the story of a budding actr...,La Land is an absolutely beautiful film with s...,french,POSITIVE,0.999891
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","The Nice Guys tries too hard to be funny, and ...",english,NEGATIVE,0.999796
15,Le Dîner de Cons,1998,The film follows the story of a group of rich ...,I didn't like this movie at all. The concept o...,french,NEGATIVE,0.999644
11,Intouchables,2011,This film tells the story of the unlikely frie...,Untouchables is an incredibly touching film wi...,french,POSITIVE,0.999877


In [17]:
# export the results to a .csv file
df.to_csv("./result/reviews_with_sentiment.csv", index=False)